In [ ]:
import os
print(os.listdir()) #list files avalilable

import numpy as np
import pandas as pd
#import the training data
train_data=pd.read_csv("../input/application_train.csv")

In [ ]:
test=pd.read_csv('../input//application_test.csv') #import test data set 
test.head() # to show first 5 rows of test data set

In [ ]:
train_data.shape

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
         # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(train_data)
missing_values.head(20)

In [ ]:
# Number of each type of column
np.where(train_data.dtypes=='object')

In [ ]:
# Number of each type of column
train_data.dtypes.value_counts()

In [ ]:
train_data.select_dtypes('object').apply(pd.Series.nunique, axis = 0)


In [ ]:
train_data.nunique()

In [ ]:
# Create a label encoder object
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_data:
    if train_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_data[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_data[col])
            # Transform both training and testing data
            train_data[col] = le.transform(train_data[col])
            test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
for i in train_data.columns:
    if train_data[i].dtypes=='object':
        print(train_data[i].value_counts())

In [ ]:
#one hot encoding
train_data= pd.get_dummies(train_data)

In [ ]:
test= pd.get_dummies(test)

In [ ]:
train_data.shape

In [ ]:
test.shape

<h1>Aligning Training and Testing Data</h1>
<body>There need to be the same features (columns) in both the training and testing data. One-hot encoding has created more columns in the training data because there were some categorical variables with categories not represented in the testing data. To remove the columns in the training data that are not in the testing data, we need to align the dataframes. First we extract the target column from the training data (because this is not in the testing data but we need to keep this information). When we do the align, we must make sure to set axis = 1 to align the dataframes based on the columns and not on the rows!</body>

In [ ]:
train_target=train_data['TARGET']

In [ ]:
train_data, test = train_data.align(test, join = 'inner', axis = 1)

In [ ]:
train_data.shape

In [ ]:
test.shape

In [ ]:
train_data['TARGET']=train_target

In [ ]:
train_data.shape

<h1>Exterior sources</h1>

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = train_data[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
poly_features= train_data[['TARGET','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]


In [ ]:
poly_features_test= test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]


In [ ]:
# to impute missing values we gonna use sklearn.imputer method 

from sklearn.preprocessing import Imputer
imputer= Imputer(strategy='median')
#poly_features['TARGET']=poly_target
poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])
# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)


# Create the polynomial object with specified degree
#poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

In [ ]:
poly_features.columns

In [ ]:
# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

In [ ]:
poly_features.to_pickle("Poly_features.csv")

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = train_data['SK_ID_CURR']
train_data_poly = train_data.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = test['SK_ID_CURR']
test_poly = test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
train_data_poly, test_poly = train_data_poly.align(test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', train_data_poly.shape)
print('Testing data with polynomial features shape:  ', test_poly.shape)

In [ ]:
train_data_poly.isnull().sum()


In [ ]:
test_poly.isnull().sum()

In [ ]:
train_data.columns

<h1>Logistic regression implementation</h1>


In [ ]:
from sklearn.preprocessing import  Imputer

# Drop the target from the training data
if 'TARGET' in train_data:
    train = train_data.drop(columns = ['TARGET'])
else:
    train = train_data.copy()
    

In [ ]:
# Feature names
features = list(train.columns)

In [ ]:
# Copy of the testing data
test_test = test.copy() # after imputation test set would be converted to numpy array which in turn would be made difficult 
#for visualization.

In [ ]:
test_test.head()

In [ ]:
train.columns

In [ ]:
#ignore_col= ['SK_ID_CURR']

In [ ]:
# Median imputation of missing values
imputer = Imputer(strategy = 'median')


#train["AMT_GOODS_PRICE"].fillna(trainX["AMT_GOODS_PRICE"].mean(), inplace = True)

In [ ]:
# Fit on the training data
imputer.fit(train)
# Transform both training and testing data
train = imputer.transform(train)


In [ ]:
test = imputer.transform(test)

In [ ]:
print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_X,test_X,train_y,test_y= train_test_split(train,train_target,test_size=0.25)

In [ ]:
# Make the model with the specified regularization parameter
Classifier = LogisticRegression(C = 0.0001)

# Train on the training data
Classifier.fit(train_X,train_y)

In [ ]:
ypred= Classifier.predict(test_X) # predicting on validation set(test_X) 

In [ ]:
from sklearn.metrics import accuracy_score
accuracy= accuracy_score(test_y,ypred)

In [ ]:
accuracy

### K fold cross validation

In [ ]:
# k fold cross validation
from sklearn.model_selection import cross_val_score
cv_score= cross_val_score(Classifier, train,train_target,cv=10)

In [ ]:
cv_score
cv_score.mean()
# after performing 10 fold cross validation cv_score is 91.9 which is similar to the accuracy_score of the logistic regresion 
#classifier

### Applying logreg on test set

In [ ]:
test_pred= Classifier.predict(test)

In [ ]:
test_pred.shape

In [ ]:
test_pred

In [ ]:
# Make predictions
# Make sure to select the second column only
#Classifier_pred = Classifier.predict_proba(test)[:, 1]
#ypred=  Classifier.predict(test)

In [ ]:
# Submission dataframe
submit = test_test[['SK_ID_CURR']]
submit['TARGET'] = test_pred

submit.head()


In [ ]:
#pred=np.apply_along_axis((lambda X: 1 if X>0.5 else 0),0,Classifier_pred)

In [ ]:
submit.to_csv("submit1")

In [ ]:
submit.to_csv("Logreg.csv", index = False)

In [ ]:
### Logistic regression performs bad on test set. completely a biased model. shouldn't be deployed.

### Boosting Algorithm

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xval, ytrain, yval = train_test_split(train_data_poly, train_target, test_size = 0.2,
                                             random_state = 1982)

In [ ]:
#total rows if we append test set to the training set. just for assumption no use in later code

train_1=pd.concat([train_data_poly, test_poly], axis= 0)
train_1.shape


### Dmarices for train, val and test set

In [ ]:
# to feed data to xgboost first training set is transformed into Dmatrix. in code below train, validation and test sets are transformed.
xgtrain = xgb.DMatrix(Xtrain, label = ytrain)
xgval = xgb.DMatrix(Xval, label = yval) 
xgtest = xgb.DMatrix(test_poly)

In [ ]:
watchlist = [(xgtrain,'train'),(xgval, 'eval')]

In [ ]:
params = {}
params["objective"] =  "binary:logistic"
params["booster"] = "gbtree"
params["max_depth"] = 7
params["eval_metric"] = 'auc'
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
params["silent"] = 1
params["seed"] = 4
params["eta"] = 0.1

plst = list(params.items())

In [ ]:
num_rounds = 500
model_cv = xgb.train(plst, xgtrain, num_rounds, evals = watchlist, early_stopping_rounds = 10, verbose_eval = True)

In [ ]:
testxg_pred = model_cv.predict(xgtest)

In [ ]:
test_id=test_test['SK_ID_CURR'] #this code is to run the above code

In [ ]:
preds = pd.DataFrame({"SK_ID_CURR": test_id, "TARGET": testxg_pred})

In [ ]:
preds.to_csv("xgb_model22aug.csv", index = False)

In [ ]:
testxg_pred

In [ ]:
#  to determine the acccuracy or any metric we need test set of target variable which we don't have with us. so I'll submit the 
# xgb_model22aug.csv to check how good the model predicts on unseen data.

### feature importance

In [ ]:
feat_imp = pd.Series(model_cv.get_fscore()).sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
feat_imp[:25].plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')
plt.show()

In [ ]:
import pickle
pickle.dump(model_cv, open("xgb_model13july.pickle.dat", "wb"))

In [ ]:
loaded_model = pickle.load(open("xgb_model13july.pickle.dat", "rb"))